In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os, time, itertools, random, datetime
import matplotlib.pyplot as plt
from matplotlib.image import imread

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorboard


In [2]:
for dirpath, dir_name, file_name in os.walk("101_food_classes_10_percent/train"):
    print(f"There are {len(dir_name)} directories and {len(file_name)} images in directory {dirpath}.")

In [3]:
train_data_path = "101_food_classes_10_percent.nosync//train"
test_data_path = "101_food_classes_10_percent.nosync//test"

train_data = image_dataset_from_directory(train_data_path,
                                          label_mode="categorical",
                                          batch_size=32,
                                          image_size=(224, 224),
                                          shuffle=True,
                                          seed=42,
                                          )
test_data = image_dataset_from_directory(test_data_path,
                                          label_mode="categorical",
                                          batch_size=32,
                                          image_size=(224, 224),
                                          shuffle=False,
                                          seed=42,
                                          )

Found 7575 files belonging to 101 classes.


2024-08-15 10:14:11.832825: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2024-08-15 10:14:11.832846: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-08-15 10:14:11.832852: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-08-15 10:14:11.832864: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-08-15 10:14:11.832884: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Found 25250 files belonging to 101 classes.


In [4]:
os.makedirs("ckpt", exist_ok=True)
checkpoint_filepath = "ckpt/checkpoint_model.weights.h5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_filepath,
    monitor='val_accuracy',
    save_best_only=True,
    save_weights_only = True
)

In [5]:
def create_tensorboard_callback_tf(dir_name, experiment_name):
    log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%y-%m-%d-%H-%M-%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir)
    print(f"Saving tensorflow callback to dir {log_dir}")
    return tensorboard_callback

In [6]:
image_augmentation = tf.keras.models.Sequential([
    tf.keras.layers.RandomFlip(mode='horizontal_and_vertical', seed=42),
    tf.keras.layers.RandomRotation(factor=(0, 0.2), fill_mode='reflect', seed=42),
    tf.keras.layers.RandomHeight(0.2),
    tf.keras.layers.RandomWidth(0.2),
    tf.keras.layers.RandomZoom(0.2),
    # tf.keras.layers.Rescaling(1./255) # rescale inputs of images between 0 and 1 . required for models like ResNet50
], name='image_augmentation')

base_model = EfficientNetB0(include_top=False, weights='imagenet')
base_model.trainable=False

inputs = tf.keras.layers.Input(shape=(224, 224, 3), name='input_layer')
x = image_augmentation(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pool_layer")(x)
outputs = tf.keras.layers.Dense(units=101, activation='softmax', name='output_layer')(x)

model = tf.keras.Model(inputs, outputs)
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

In [7]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ image_augmentation (Sequential) │ (None, None, None, 3)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, None, None,     │     4,049,571 │
│                                 │ 1280)                  │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pool_layer       │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 101)            │       129,381 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,178,952 (15.94 MB)

 Trainable params: 129,381 (505.39 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [8]:
model1_hist = model.fit(train_data.repeat(),
                        epochs=5,
                        steps_per_epoch=len(train_data),
                        validation_data=test_data.repeat(),
                        validation_steps=int(0.15 * len(test_data)), # validate only 15% of the overall test data.
                        callbacks=[create_tensorboard_callback_tf("EfficientNetB0", "101_food_10_percent"), model_checkpoint_callback])

Saving tensorflow callback to dir EfficientNetB0/101_food_10_percent/24-08-15-10-14-21
Epoch 1/5


2024-08-15 10:14:22.699048: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


237/237 ━━━━━━━━━━━━━━━━━━━━ 235s 976ms/step - accuracy: 0.1224 - loss: 4.1256 - val_accuracy: 0.3414 - val_loss: 2.9510
Epoch 2/5
237/237 ━━━━━━━━━━━━━━━━━━━━ 318s 1s/step - accuracy: 0.3854 - loss: 2.7204 - val_accuracy: 0.3665 - val_loss: 2.5188
Epoch 3/5
237/237 ━━━━━━━━━━━━━━━━━━━━ 356s 2s/step - accuracy: 0.4478 - loss: 2.3312 - val_accuracy: 0.5432 - val_loss: 1.8807
Epoch 4/5
237/237 ━━━━━━━━━━━━━━━━━━━━ 415s 2s/step - accuracy: 0.4896 - loss: 2.0914 - val_accuracy: 0.5201 - val_loss: 1.9428
Epoch 5/5
237/237 ━━━━━━━━━━━━━━━━━━━━ 521s 2s/step - accuracy: 0.5239 - loss: 1.9549 - val_accuracy: 0.5988 - val_loss: 1.6599


In [9]:
for index, layer in enumerate(model.layers):
    print(index, layer.name, layer.trainable)

0 input_layer True
1 image_augmentation True
2 efficientnetb0 False
3 global_average_pool_layer True
4 output_layer True


In [10]:
model.evaluate(test_data)

790/790 ━━━━━━━━━━━━━━━━━━━━ 225s 284ms/step - accuracy: 0.4856 - loss: 2.0337


[1.892327070236206, 0.5178217887878418]

In [12]:
base_model.trainable =True
for layer in base_model.layers[:-5]: # set top 5 layers trainable.
    layer.trainable = False

In [14]:
# recompile the model when ever we make any changes to the model.
model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(1e-4), metrics=["accuracy"]) 


In [15]:
model1_hist_fine_tuned = model.fit(train_data.repeat(),
                                    epochs=10,
                                    steps_per_epoch=len(train_data),
                                    validation_data=test_data.repeat(),
                                    validation_steps=int(0.15 * len(test_data)),
                                    initial_epoch=model1_hist.epoch[-1])

Epoch 5/10
237/237 ━━━━━━━━━━━━━━━━━━━━ 1894s 8s/step - accuracy: 0.5261 - loss: 1.9428 - val_accuracy: 0.4762 - val_loss: 2.0803
Epoch 6/10
237/237 ━━━━━━━━━━━━━━━━━━━━ 1384s 6s/step - accuracy: 0.5772 - loss: 1.7328 - val_accuracy: 0.5011 - val_loss: 1.9183
Epoch 7/10
188/237 ━━━━━━━━━━━━━━━━━━━━ 3:32 4s/step - accuracy: 0.5903 - loss: 1.6327

In [ ]:
model.evaluate(test_data)

In [ ]:
pred_prob = model.evaluate(test_data)
pred_Classes = pred_prob.argmax(axis=1)